In [55]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
import statsmodels.api as sm

import warnings
warnings.filterwarnings('ignore')

import nbimporter
import spambase_dataset_loader_nb
import spambase_model_helper_nb

In [56]:
loader = spambase_dataset_loader_nb.SpambaseDatasetLoader()

loader.load_dataset()

init Loader notebook
Loading dataset.
Loading spambase_all.csv completed.
Loading spambase_balance.csv completed.


In [57]:
spambase_balance = loader.get_balance_dataset()
data = spambase_balance.copy()
spambase_balance.head()

get_balance_data


,target_spam_nonspam,word_freq_make,word_freq_address,word_freq_all,word_freq_3d,word_freq_our,word_freq_over,word_freq_remove,word_freq_internet,word_freq_order,...,word_freq_conference,char_freq_;,char_freq_(,char_freq_[,char_freq_!,char_freq_$,char_freq_#,capital_run_length_average,capital_run_length_longest,capital_run_length_total
0,1,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,...,0.0,0.00,0.000,0.0,0.778,0.000,0.000,3.756,61,278
1,1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,...,0.0,0.00,0.132,0.0,0.372,0.180,0.048,5.114,101,1028
2,1,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,...,0.0,0.01,0.143,0.0,0.276,0.184,0.010,9.821,485,2259
3,1,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,...,0.0,0.00,0.137,0.0,0.137,0.000,0.000,3.537,40,191
4,1,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,...,0.0,0.00,0.135,0.0,0.135,0.000,0.000,3.537,40,191


In [80]:
selected_columns = data.columns
selected_columns = selected_columns[1:].values # remove the label

X = data.iloc[:,1:].values
y = data.iloc[:,0].values

In [81]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=101)

## Feature Selection using Logit

In [82]:
import statsmodels.api as sm

# building the model and fit the model using the training data
statsmodels_lr = sm.Logit(y_train, X_train).fit()

print(statsmodels_lr.summary())

Optimization terminated successfully.
         Current function value: 0.209908
         Iterations 15
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                 2900
Model:                          Logit   Df Residuals:                     2843
Method:                           MLE   Df Model:                           56
Date:                Thu, 27 Jul 2023   Pseudo R-squ.:                  0.6972
Time:                        06:57:36   Log-Likelihood:                -608.73
converged:                       True   LL-Null:                       -2010.1
Covariance Type:            nonrobust   LLR p-value:                     0.000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x1            -0.7281      0.281     -2.594      0.009      -1.278      -0.178
x2            -0.2198      0

In [84]:
x = X
Y = y
sl = 0.05
numVars = len(x[0])
columns = selected_columns

for i in range(0, numVars):
    regressor_Logit = sm.Logit(Y, x).fit()
    maxVar = max(regressor_Logit.pvalues).astype(float)
    print('******* i: {}, maxVar: {}, sl: {}'.format(i, maxVar, sl))
    if maxVar > sl:        
        for j in range(0, numVars - i):
            #print('p_values: {}'.format(regressor_OLS.pvalues[j].astype(float)))
            if (regressor_Logit.pvalues[j].astype(float) == maxVar):                
                x = np.delete(x, j, 1)
                columns = np.delete(columns, j)
                print(columns.shape)
        #print(regressor_OLS.summary())

print(columns)

Optimization terminated successfully.
         Current function value: 0.206374
         Iterations 15
******* i: 0, maxVar: 0.9943655003213443, sl: 0.05
(56,)
Optimization terminated successfully.
         Current function value: 0.206374
         Iterations 15
******* i: 1, maxVar: 0.8938749201903528, sl: 0.05
(55,)
Optimization terminated successfully.
         Current function value: 0.206376
         Iterations 15
******* i: 2, maxVar: 0.8842928047216136, sl: 0.05
(54,)
Optimization terminated successfully.
         Current function value: 0.206379
         Iterations 15
******* i: 3, maxVar: 0.8552950041938079, sl: 0.05
(53,)
Optimization terminated successfully.
         Current function value: 0.206384
         Iterations 15
******* i: 4, maxVar: 0.7036909957574553, sl: 0.05
(52,)
Optimization terminated successfully.
         Current function value: 0.206402
         Iterations 15
******* i: 5, maxVar: 0.6982068742207083, sl: 0.05
(51,)
Optimization terminated successfully.
  

In [62]:
columns.size

34

## Pearson Correlation

In [77]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

corr = spambase_balance.corr()
cor_target = abs(corr['target_spam_nonspam'])
#Selecting highly correlated features
relevant_features = cor_target[cor_target>0.2]
list4 = relevant_features.keys().to_list()

In [78]:
list4

['target_spam_nonspam',
 'word_freq_all',
 'word_freq_our',
 'word_freq_over',
 'word_freq_remove',
 'word_freq_order',
 'word_freq_receive',
 'word_freq_free',
 'word_freq_business',
 'word_freq_email',
 'word_freq_you',
 'word_freq_your',
 'word_freq_000',
 'word_freq_money',
 'word_freq_hp',
 'word_freq_hpl',
 'word_freq_george',
 'char_freq_!',
 'char_freq_$',
 'capital_run_length_total']

In [76]:
len(list4)

20

### Feature Selection using Chi-Square

In [91]:
X = data.iloc[:,1:]
y = data.iloc[:,0].values

chi2_features = SelectKBest(chi2, k=34)
X_kbest_features = chi2_features.fit_transform(X, y)
mask=chi2_features.get_support()
new_features=[]
for bool, feature in zip(mask, X.columns):
    if (bool):
        new_features.append(feature)
list3=new_features 
list3

['word_freq_3d',
 'word_freq_our',
 'word_freq_remove',
 'word_freq_internet',
 'word_freq_addresses',
 'word_freq_free',
 'word_freq_business',
 'word_freq_email',
 'word_freq_you',
 'word_freq_credit',
 'word_freq_your',
 'word_freq_font',
 'word_freq_000',
 'word_freq_money',
 'word_freq_hp',
 'word_freq_hpl',
 'word_freq_george',
 'word_freq_650',
 'word_freq_lab',
 'word_freq_labs',
 'word_freq_telnet',
 'word_freq_data',
 'word_freq_85',
 'word_freq_1999',
 'word_freq_pm',
 'word_freq_meeting',
 'word_freq_project',
 'word_freq_re',
 'word_freq_edu',
 'char_freq_!',
 'char_freq_$',
 'capital_run_length_average',
 'capital_run_length_longest',
 'capital_run_length_total']

In [92]:
len(list3)

34

In [93]:
new_features = X.columns[mask]

In [94]:
len(new_features)

34

In [95]:
new_features

Index(['word_freq_3d', 'word_freq_our', 'word_freq_remove',
       'word_freq_internet', 'word_freq_addresses', 'word_freq_free',
       'word_freq_business', 'word_freq_email', 'word_freq_you',
       'word_freq_credit', 'word_freq_your', 'word_freq_font', 'word_freq_000',
       'word_freq_money', 'word_freq_hp', 'word_freq_hpl', 'word_freq_george',
       'word_freq_650', 'word_freq_lab', 'word_freq_labs', 'word_freq_telnet',
       'word_freq_data', 'word_freq_85', 'word_freq_1999', 'word_freq_pm',
       'word_freq_meeting', 'word_freq_project', 'word_freq_re',
       'word_freq_edu', 'char_freq_!', 'char_freq_$',
       'capital_run_length_average', 'capital_run_length_longest',
       'capital_run_length_total'],
      dtype='object')